In [7]:
import requests  
from lxml import etree 
import pickle
import os
from IPython.core.display import display, HTML
import timeit
import jieba

class MySearcherC10V0:
    """
    第九次课升级的搜索类版本：
    1、多词摘要和高亮的处理
    """
    def __init__(self, scale=1):
        self.docs = []
        self.load_data()
        self.docs *= scale
        self.cache = {}
        self.vocab = set()
        self.lower_preprocess()
        self.build_cache()
        jieba.load_userdict('dict.txt')
    
    def build_cache(self):
        doc_id = 0
        for doc in self.docs:
            doc_word_set = set()
            for word in jieba.cut_for_search(
                doc[3]
            ):
                if word not in doc_word_set:
                    result_item = doc_id
                    if word not in self.cache:
                        self.cache[word] = set([result_item])
                    else:
                        self.cache[word].add(result_item)
                    self.vocab.add(word)
                    doc_word_set.add(word)
            doc_id += 1
    
    def search(self, query):
        result = None
        for keyword in jieba.cut(query.lower()):
            if keyword in self.cache:
                if result is None:
                    result = self.cache[keyword]
                else:
                    result = result & self.cache[keyword]
            else:
                result = set([])
                break
                
        if result is None:
            result = set([])
        
        sorted_result = self.rank(query, result)
        return sorted_result
                    
    def lower_preprocess(self):
        for doc_id in range(len(self.docs)):
            self.docs[doc_id].append(
                (self.docs[doc_id][1] 
                 + ' ' 
                 + self.docs[doc_id][2]).lower()
            )
    
    def simple_test(self):
        assert(len(self.search('tiktok')) > 1)
    
    def load_data(self):
        data_filename = 'news_list.dat'
        if os.path.exists(data_filename):
            with open(data_filename,'rb') as f:
                self.docs += pickle.load(f)
#                 self.docs = self.docs + pickle.load(f)
        else:
            url = 'http://news.163.com/special/0001386F/rank_tech.html'  
            headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36 Edg/85.0.564.63'}
            r = requests.get(url, headers=headers)  
            sel = etree.HTML(r.text) 
            link_set = set()
            news_list = []
            count = 0
            for item in sel.xpath('//td/a'):  
                title = item.text
                link = item.attrib['href']
            #     print(link, title)
                if link not in link_set:
                    r = requests.get(link, headers=headers)  
                    sel = etree.HTML(r.text)  
                    text_block = sel.xpath('//div[@id="endText"]') 
                #     print(''.join(text_block[0].itertext()))
                    if text_block:
                        content = ''.join(text_block[0].xpath('./p/text()'))
                        title = sel.xpath('//h1/text()')[0]
                        self.docs.append([link, title, content])
                    link_set.add(link)
                count += 1
                if count % 15 == 0:
                    print(count, 'processed.')
            with open(data_filename,'wb') as f:
                pickle.dump(self.docs, f)
    
    def highlight(self, item, query, sidelen=12):
        result = ''
        positions = []
        content_lower = item[2].lower()
        word_start_map = []
        word_end_map = []
        last_word_end = -1
        query_words = list(jieba.cut(query))
        for keyword in query_words:
            idx = content_lower.find(keyword.lower())
            positions.append(idx)

        for keyword in jieba.cut(content_lower):
            cur_word_start = last_word_end + 1
            cur_word_end = cur_word_start + len(keyword) - 1
            for i in range(cur_word_start, cur_word_end + 1):
                word_start_map.append(cur_word_start)
                word_end_map.append(cur_word_end)
            last_word_end = cur_word_end

    #     print(word_start_map)
    #     print(word_end_map)
    #     print(positions)
        positions.sort()
    #     print(positions)
        segments = []
        i = 0
        while i < len(positions):
            start_pos = max(positions[i] - sidelen, 0)
            end_pos = min(positions[i] + sidelen, len(content_lower) - 1)
            while (i < len(positions) - 1 
                and positions[i+1] - positions[i] <= 2 * sidelen):
                end_pos = min(positions[i+1] + sidelen, len(content_lower) - 1)
                i += 1  
            start_ddd = '...' if start_pos > 0 else ''
            end_ddd = '...' if end_pos < len(content_lower) else ''
#             print(len(content_lower), end_pos)
            segments.append(start_ddd 
                            + item[2][word_start_map[start_pos]:word_end_map[end_pos] + 1]
                            + end_ddd)
            i += 1
    #     print(segments)
        result = text = item[1] + '<br/>' + ''.join(segments)
        text_lower = text.lower()
        for keyword in query_words:
    #         print(keyword)
            idx = text_lower.find(keyword.lower())
            if idx >= 0:
                ori_word = text[idx:idx+(len(keyword))]
    #             print(ori_word)
                result = result.replace(ori_word, '<span style="color:red";>{}</span>'.format(ori_word))
        return result
    
    def render_search_result(self, query):
        count = 0
        for item in self.search(query)[:10]:
            count += 1
            display(HTML('{} [{}] {}'.format(count, item[1], 
                self.highlight(self.docs[item[0]], query))))
#             print(self.highlight(self.docs[item[0]], query))
    
    def rank(self, query, result_set):
        result = []
        for doc_id in result_set:
            result.append([doc_id, 
                self.score(self.docs[doc_id],
                          query)])
        result.sort(key=lambda x: x[1], reverse=True)
        return result       
    
    def score(self, item, query):
        score = 0
        #todo cut
        for keyword in jieba.cut(query):
            score += item[1].lower().count(keyword.lower()) * 5 \
                      + item[2].lower().count(keyword.lower()) * 3
        return score

In [8]:
searcher_v0 = MySearcherC10V0()

In [43]:
searcher_v0.render_search_result('中国华为手机')

In [26]:
class MySearcherC10V1(MySearcherC10V0):
    """
    在score函数里添加停用词
    """
    def __init__(self, scale=1):
        self.docs = []
        self.load_data()
        self.docs *= scale
        self.cache = {}
        self.vocab = set()
        self.lower_preprocess()
        self.build_cache()
        self.stopwords = set()
        self.init_stopwords()
        jieba.load_userdict('dict.txt')
        
    def init_stopwords(self):
        with open('stopwords.txt', 'r', encoding='UTF-8') as f:
            stopword_list = f.read().split()
        self.stopwords = set(stopword_list)
        
    def score(self, item, query):
        score = 0
        #todo cut
        for keyword in jieba.cut(query):
            weight = 1
            if keyword in self.stopwords:
                weight = 0.2
                
            score += (item[1].lower().count(keyword.lower()) * 5 \
                      + item[2].lower().count(keyword.lower()) * 3) * weight
        return score

In [27]:
searcherv1 = MySearcherC10V1()

In [28]:
searcherv1.render_search_result('以及手机')

In [19]:
len(searcherv1.stopwords)

745

In [14]:
with open('stopwords.txt', 'r', encoding='UTF-8') as f:
    stopword_list = f.read().split()

In [16]:
stopword_list[-10:]

['非独', '靠', '顺', '顺着', '首先', '！', '，', '：', '；', '？']

In [40]:
class MySearcherC10V2(MySearcherC10V0):
    """
    用文档频率(DF)对词进行加权
    """
    def __init__(self, scale=1):
        self.docs = []
        self.load_data()
        self.docs *= scale
        self.cache = {}
        self.vocab = set()
        self.lower_preprocess()
        self.df = {}
        self.build_cache()
        jieba.load_userdict('dict.txt')
        
    def build_cache(self):
        doc_id = 0
        for doc in self.docs:
            doc_word_set = set()
            for word in jieba.cut_for_search(
                doc[3]
            ):
                if word not in doc_word_set:
                    result_item = doc_id
                    if word not in self.cache:
                        self.cache[word] = set([result_item])
                    else:
                        self.cache[word].add(result_item)
                    self.vocab.add(word)
                    doc_word_set.add(word)
                    
                    if word in self.df:
                        self.df[word] += 1
                    else:
                        self.df[word] = 1
            doc_id += 1
            
    def score(self, item, query):
        score = 0
        #todo cut
        for keyword in jieba.cut(query):
            score += (item[1].lower().count(keyword.lower()) * 5 \
                      + item[2].lower().count(keyword.lower()) * 3) \
                     / len(item[2]) / (0.01 + self.df[keyword])
        return score

In [41]:
searcherv2 = MySearcherC10V2()

In [42]:
searcherv2.render_search_result('中国华为手机')